In [ ]:
import cv2 
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
import pickle 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train_path = 'C:\FileD\JupyterNotebook\BengkelKodingCV\dataset'
training_names = os.listdir(train_path)
image_paths = []
image_classes = []
class_id = 0

def imglist(path):    
    return [os.path.join(path, f) for f in os.listdir(path)]

for training_name in training_names:
    dir = os.path.join(train_path, training_name) # Menggabungkan train_path dan training_name; Dataset/Putri, Dataset/Kirei, Dataset/Yudha
    """
    print(dir)
    D:/Kuliah/Riset/Bengkod/Studi_kasus1/Dataset/Aaron_Peirsol
    D:/Kuliah/Riset/Bengkod/Studi_kasus1/Dataset/Abdoulaye_Wade
    D:/Kuliah/Riset/Bengkod/Studi_kasus1/Dataset/Abdullah
    """
    class_path = imglist(dir)
    """
    print(class_path)
    codingan untuk melihat seluruh gambar disetiap folder, contoh seperti dibawah
    'D:/Kuliah/Riset/Bengkod/Studi_kasus1/Dataset/Aaron_Peirsol\\Aaron_Peirsol_0001.jpg'
    """
    image_paths+=class_path
    """
    print(image_paths)
    menggabungkan semua gambar pada satu file array
    'D:/Kuliah/Riset/Bengkod/Studi_kasus1/Dataset/Aaron_Peirsol\\Aaron_Peirsol_0001.jpg', 
    'D:/Kuliah/Riset/Bengkod/Studi_kasus1/Dataset/Aaron_Peirsol\\Aaron_Peirsol_0002.jpg', 
    'D:/Kuliah/Riset/Bengkod/Studi_kasus1/Dataset/Aaron_Peirsol\\Aaron_Peirsol_0003.jpg', 
    'D:/Kuliah/Riset/Bengkod/Studi_kasus1/Dataset/Aaron_Peirsol\\Aaron_Peirsol_0004.jpg', 
    'D:/Kuliah/Riset/Bengkod/Studi_kasus1/Dataset/Abdoulaye_Wade\\Abdoulaye_Wade_0001.jpg', 
    'D:/Kuliah/Riset/Bengkod/Studi_kasus1/Dataset/Abdoulaye_Wade\\Abdoulaye_Wade_0002.jpg', 
    'D:/Kuliah/Riset/Bengkod/Studi_kasus1/Dataset/Abdoulaye_Wade\\Abdoulaye_Wade_0003.jpg', 
    'D:/Kuliah/Riset/Bengkod/Studi_kasus1/Dataset/Abdoulaye_Wade\\Abdoulaye_Wade_0004.jpg', 
    'D:/Kuliah/Riset/Bengkod/Studi_kasus1/Dataset/Abdullah\\Abdullah_0001.jpg', 
    'D:/Kuliah/Riset/Bengkod/Studi_kasus1/Dataset/Abdullah\\Abdullah_0002.jpg', 
    'D:/Kuliah/Riset/Bengkod/Studi_kasus1/Dataset/Abdullah\\Abdullah_0003.jpg', 
    'D:/Kuliah/Riset/Bengkod/Studi_kasus1/Dataset/Abdullah\\Abdullah_0004.jpg'
    """
    image_classes+=[class_id]*len(class_path)
    """
    print(image_classes)
    menandakan label pada setiap gambar
    """
    class_id+=1
    """
    print(class_id)
    menandakan id setiap folder
    """
    

# img = cv2.imread('Dataset/Aaron_Peirsol/Aaron_Peirsol_0001.jpg')
# imgGray = cv2.cvtColor(img, cv.COLOR_BGR2GRAY)
# sift = cv2.SIFT_create()
# kp = sift.detect(imgGray, None)
# img = cv2.drawKeypoints(imgGray, kp, img)
# cv2.imshow('gambar1', img)


In [ ]:
# Inisialisasi variabel untuk menyimpan deskriptor dari Algoritma SIFT
des_list = []

# Buat Fitur Ekstraksi dan Objek Deteksi Keypoints
# keypoints merupakan fitur untuk menandakan informasi penting yang diekstraksi
sift = cv2.SIFT_create()

for image_path in image_paths:
    im = cv2.imread(image_path) # Membaca Citra berdasarkan PATH yang telah dibuat sebelumnya
    kpts, des = sift.detectAndCompute(im, None) # Menjelaskan lokasi keypoints
    des_list.append((image_path, des))   # Menyimpan fitur yang telah dideteksi kedalam variabel des_list=[]
    # Kumpulan image_path dan keypoints pada des_list

In [ ]:
descriptors = des_list[0][1]

for image_path, descriptor in des_list[1:]:
    descriptors = np.vstack((descriptors, descriptor)) #vstack menumpuk dua layer gambar, 1 gambar polos, 2 gambar keypoints

descriptors_float = descriptors.astype(float)  # K-Means hanya bekerja pada tipe data float, Convert descriptor ke float

# Buat histogram
plt.hist(descriptors_float, bins=50, alpha=0.5, edgecolor='black')

# Atur label sumbu x dan y
plt.xlabel('Nilai Deskriptor')
plt.ylabel('Frekuensi')

# Tampilkan histogram
plt.show()

In [ ]:
# Gunakan K-Means untuk Melakukan perhitungan Histogram (BoVW)
# vq : Vector Quantization, salah satu modul SciPy fokus pada klastering dan analisis vektor
from scipy.cluster.vq import kmeans, vq 
k = 150  

voc, variance = kmeans(descriptors_float, k, 1) 
#voc hasil dari klastering K-Means
#variance merupakan penyebaran data dalam setiap kelompok yang dapat membberikan informasi tentang seberapa baik K-Means

im_features = np.zeros((len(image_paths), k), "float32")
#membuat matrix berukuran panjang data set dikali parameter k dengan nilai 0 semua 

for i in range(len(image_paths)):
    words, distance = vq(des_list[i][1],voc)
    for w in words:
        im_features[i][w] += 1


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(im_features, image_classes, test_size=0.2, random_state=42, stratify=image_classes)
#random state : di acak
#stratify=image_classes : membagi data acak sama rata

In [ ]:
from sklearn.svm import SVC
clf = SVC(kernel='linear', C=1.0, max_iter=500)
# membuat model SVM
# SVM dengan kernel linear krn ringan
# C : ada deh
# max_iter : berapa kali algoritma mempelajari dataset sebanyak 500 kali

clf.fit(X_train, np.array(y_train))
# memberikan dataset pada model
# X adalah fitur , Y adalah label

# Lakukan prediksi pada data uji
y_pred = clf.predict(X_test)

# Mencetak laporan klasifikasi
report = classification_report(y_test, y_pred)
# pada fungsi classification_report : menghasilkan precision, recall, f1-score, support
# precision (presisi) : mengukur sejauh mana model berhasil mengidentifikasi kelas positif (asli) dari seluruh prediksi positif
# recall (sensitivitas) : berguna menemukan semua kasus positif yang sebenarnya dalam dataset
# f1-score : mengukur keeseimbangan antara precision d|an recall, memberi informasi
print(report)

#Simpan Model Sistem Klasifikasi kedalam bentuk Pickel
filename = "svm_model_SIFT.pkl"
with open(filename, "wb") as f:
    pickle.dump(clf, f)

In [ ]:
# Mencetak confusion matrix
print("Confusion Matrix:")

# Membuat confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

# Memplot confusion matrix
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")

plt.xlabel("Predicted Labels")
plt.ylabel("True labels")
plt.title("Confusion Matrix")
plt.show()